In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [1]:
import ee
import os
import geopandas as gpd
import pandas as pd

In [4]:
try:
    ee.Initialize()
except:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project='potential-evap')

In [10]:
print(len(os.listdir("/content/GAGES_smooth_shp")))

46610


In [5]:
bounds = None
def gen_timeseries(img,band="precipitation"):
  img = img.select(band)
  scale = img.projection().nominalScale()
  stats = img.reduceRegion(ee.Reducer.mean(),bounds,scale)
  dt = img.date().format("YYYY-MM-dd")
  return img.set(stats.combine({"date_str":dt}))

In [11]:
# TEST SHAPES LIST
shapefiles_list = []
finished_basins = []
path = "GAGES_smooth_shp"
finished_path = "Precip_TS"

for file in os.listdir(finished_path):
  finished_basins.append(file[-12:-4])


for file in os.listdir(path):
  if file.endswith('.shp'):
    file_path = os.path.join(path,file)
    basin_no = file_path[-12:-4] #string
    if basin_no not in finished_basins:
      shapefiles_list.append(file_path)

print(len(shapefiles_list))

0


In [ ]:
# check output for finished stuff

In [10]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate("1999-01-01","2023-01-01")
basins_out_of_bounds = []
# for i in range(1):
for i in range(len(shapefiles_list)):
# for i in range(100):
  shapefile = gpd.read_file(shapefiles_list[i])
  # print(shapefile)

  #---This block converts the shapefiles to ee.features and then to a geometry bounding box of the shapefile---------
  features = []
  for j in range(shapefile.shape[0]):
    geom = shapefile.iloc[j:j+1,:]
    jsonDict = eval(geom.to_json())
    geojsonDict = jsonDict['features'][0]
    features.append(ee.Feature(geojsonDict))

  fc = ee.FeatureCollection(features)
  bounds = fc.geometry().bounds()
  #---End of block--------------

  chirps_ts = chirps.map(gen_timeseries) # the mean precip is over the bounding box, not just the polygon
  cols = chirps_ts.reduceColumns(ee.Reducer.toList().repeat(2),["precipitation","date_str"])
  precip, timestamp = cols.getInfo()["list"]
  if len(precip) > 0:
    df = pd.DataFrame({"date":timestamp, "precipitation":precip},index=pd.to_datetime(timestamp))
    file_PathName = 'Precip_TS/' + 'basin_' + shapefiles_list[i][-12:-4] + '.csv'
    print(i, '. YES \t', shapefiles_list[i][-12:-4])
    df.to_csv(path_or_buf=file_PathName,index=False)
    # print("exported!")
  else:
    df = pd.DataFrame(data={})
    file_PathName = 'Precip_TS/' + 'basin_' + shapefiles_list[i][-12:-4] + '.csv'
    df.to_csv(path_or_buf=file_PathName,index=False)
    print(i, '. NO \t', shapefiles_list[i][-12:-4])

# df.plot.area(legend=False,ylabel="Precipitation [mm]");

0 . YES 	 03126000
1 . YES 	 02411000
2 . YES 	 11098500
3 . YES 	 05432695
4 . YES 	 14166000
5 . YES 	 08064800
6 . YES 	 14158850
7 . YES 	 02400680
8 . YES 	 14162500
9 . YES 	 01420000
10 . YES 	 11151700
11 . NO 	 15200000
12 . YES 	 02479431
13 . YES 	 09424450
14 . YES 	 08164800
15 . YES 	 01524500
16 . YES 	 13177800
17 . YES 	 04138500
18 . YES 	 01520000
19 . YES 	 10149500
20 . YES 	 05382325
21 . YES 	 11441900
22 . YES 	 06312500
23 . YES 	 14010800
24 . YES 	 11274538
25 . YES 	 11119750
26 . YES 	 06447000
27 . YES 	 07170000
28 . YES 	 05485605
29 . YES 	 06212500
30 . YES 	 12393300
31 . YES 	 08012000
32 . YES 	 02339500
33 . YES 	 08268700
34 . YES 	 02468500
35 . YES 	 06216000
36 . YES 	 09270500
37 . YES 	 50076000
38 . YES 	 14163150
39 . YES 	 16419500
40 . YES 	 10351650
41 . YES 	 06424000
42 . YES 	 50115000
43 . YES 	 50111500
44 . YES 	 08171000
45 . YES 	 06275000
46 . YES 	 02388320
47 . YES 	 09213500
48 . YES 	 08075500
49 . YES 	 07013000
50 . YES 	 

#Appending Dates

In [ ]:
dates_df = pd.read_csv('/content/drive/MyDrive/IDeA Lab/The Equation/Precipitation/precip_dates.csv')
dates_df

,Date
0,1999-01-01
1,1999-01-02
2,1999-01-03
3,1999-01-04
4,1999-01-05
...,...
7300,2018-12-27
7301,2018-12-28
7302,2018-12-29
7303,2018-12-30


In [ ]:
basin_df = pd.read_csv('/content/drive/MyDrive/IDeA Lab/The Equation/Precipitation/Basin_Precip_TS/basin_1104200.csv')
basin_df

,precipitation
0,11.817496
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
7300,0.000000
7301,0.000000
7302,0.336493
7303,0.046856


In [ ]:
#TEST
precip_with_dates_df = pd.concat([basin_df,dates_df],axis=1)
precip_with_dates_df = precip_with_dates_df.set_index('Date')
precip_with_dates_df

,precipitation
Date,
1999-01-01,12.676548
1999-01-02,0.000000
1999-01-03,0.000000
1999-01-04,0.000000
1999-01-05,0.000000
...,...
2018-12-27,0.000000
2018-12-28,0.000000
2018-12-29,0.000000
